In [54]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [55]:
df = pd.read_csv('data/GHDomains.csv')

In [56]:
# excluded_labels = ['Web libraries and frameworks', 'Documentation']
# df = df[~df['Domain'].isin(excluded_labels)]
df = df[df['Status']==True].reset_index(drop=True)
df

,Stars,Forks,Description,Readme,Languages,Licence,Releases,Status,Contents,Name,Domain,clean_readme,english_readme,clean_description,english_description
0,323943.0,25410.0,freeCodeCamp.org's open source codebase and cu...,![freeCodeCamp.org Social Banner](https://s3.a...,"['JavaScript', 'CSS', 'Shell', 'HTML', 'EJS', ...","BSD 3-Clause ""New"" or ""Revised"" License",0.0,True,"['.dockerignore', '.editorconfig', '.eslintign...",freeCodeCamp/freeCodeCamp,Documentation,freeCodeCamp.org's open-source codebase and cu...,1.000000,freeCodeCamp.org's open source codebase and cu...,1.0
1,150012.0,71296.0,"The most popular HTML, CSS, and JavaScript fra...","<p align=""center"">\n <a href=""https://getboot...","['JavaScript', 'HTML', 'SCSS', 'CSS', 'PowerSh...",MIT License,69.0,True,"['.babelrc.js', '.browserslistrc', '.bundlewat...",twbs/bootstrap,Web libraries and frameworks,"Bootstrap Sleek, intuitive, and powerful fron...",0.979592,"The most popular HTML, CSS, and JavaScript fra...",1.0
2,96800.0,22341.0,"Bring data to life with SVG, Canvas and HTML. ...","# D3: Data-Driven Documents\n\n<a href=""https:...",['JavaScript'],"BSD 3-Clause ""New"" or ""Revised"" License",167.0,True,"['.gitignore', 'API.md', 'CHANGES.md', 'ISSUE_...",d3/d3,Web libraries and frameworks,D3: Data-Driven Documents D3 (or D3.js) is a J...,1.000000,"Bring data to life with SVG, Canvas and HTML. ...",1.0
3,168294.0,33061.0,"A declarative, efficient, and flexible JavaScr...",# [React](https://reactjs.org/) &middot; [![Gi...,"['JavaScript', 'HTML', 'CSS', 'C++', 'TypeScri...",MIT License,96.0,True,"['.circleci', '.codesandbox', '.editorconfig',...",facebook/react,Web libraries and frameworks,React React is a JavaScript library for buildi...,1.000000,"A declarative, efficient, and flexible JavaScr...",1.0
4,59608.0,28135.0,AngularJS - HTML enhanced for web apps!,AngularJS [![CircleCI](https://circleci.com/gh...,"['JavaScript', 'HTML', 'Shell', 'PHP', 'CSS']",MIT License,0.0,True,"['.circleci', '.editorconfig', '.eslintignore'...",angular/angular.js,Web libraries and frameworks,AngularJS AngularJS lets you write client-side...,1.000000,AngularJS - HTML enhanced for web apps!,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,1850.0,349.0,This project aims to provide a working page fl...,# Changes:\n\n * Made clickable views like a ...,['Java'],NaN,0.0,True,"['.gitignore', 'README.md', 'build.gradle', 'f...",Yalantis/FlipViewPager.Draco,Non-web libraries and frameworks,Changes: Made clickable views like a button cl...,0.937500,This project aims to provide a working page fl...,1.0
4960,1780.0,178.0,JavaScript Client-Side Cookie Manipulation Lib...,# Cookies.js\n\nCookies.js is a small client-s...,"['JavaScript', 'CSS', 'HTML']",The Unlicense,10.0,True,"['.gitignore', 'CHANGELOG.md', 'README.md', 'U...",ScottHamper/Cookies,Web libraries and frameworks,Cookies.js Cookies.js is a small client-side j...,1.000000,JavaScript Client-Side Cookie Manipulation Lib...,1.0
4961,3138.0,300.0,The earliest versions of the very first c comp...,legacy-cc\n=========\n\nThe earliest versions ...,"['C', 'Assembly']",NaN,0.0,True,"['Caldera-license.pdf', 'README.md', 'last1120...",mortdeus/legacy-cc,Software tools,legacy-cc The earliest versions of the very fi...,1.000000,The earliest versions of the very first c comp...,1.0
4962,1668.0,441.0,A clone of the UIImagePickerController using t...,# ELCImagePickerController\n\n*A clone of the ...,"['Objective-C', 'Ruby']",NaN,1.0,True,"['.gitignore', 'Classes', 'Default-568h@2x.png...",B-Sides/ELCImagePickerController,Non-web libraries and frameworks,ELCImagePickerController A clone of the UIImag...,1.000000,A clone of the UIImagePickerController using t...,1.0


In [57]:
text = df['clean_description'] + df['clean_readme']

to_drop = []
for i in text.index:
    if not isinstance(text.loc[i], str):
        to_drop.append(i)

In [58]:
text = text.drop(to_drop).reset_index(drop=True)
y = df['Domain'].drop(to_drop).reset_index(drop=True)

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text, y, test_size=0.1, random_state=42)

In [62]:
X_train.to_csv('data/X_train.csv')
X_test.to_csv('data/X_test.csv')
y_train.to_csv('data/y_train.csv')
y_test.to_csv('data/y_test.csv')

In [74]:
X_train['len'] = X_train.apply(len)

In [81]:
X_train['len'].mean()

5694.091405184175

In [63]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [71]:
embeddings = model.encode(X_train.to_numpy())

In [72]:
embeddings.tofile('data/X_train_embeddings.dat')

In [69]:
embeddings = model.encode(X_test.to_numpy())

In [70]:
embeddings.tofile('data/X_test_embeddings.dat')

In [91]:
embeddings.shape

(4398, 384)

In [68]:
len(X_train)

4398

In [67]:
len(embeddings)

4398

In [4]:
df['Domain'].value_counts()

Web libraries and frameworks        1528
Non-web libraries and frameworks    1432
Software tools                       966
Application software                 430
Documentation                        427
System software                      181
Name: Domain, dtype: int64

In [5]:
mapping = {}
labels = df['Domain'].unique()
for cat, num in zip(labels, range(len(labels))):
    mapping[cat] = num
mapping


{'Documentation': 0,
 'Web libraries and frameworks': 1,
 'Software tools': 2,
 'System software': 3,
 'Non-web libraries and frameworks': 4,
 'Application software': 5}

In [6]:
y = df['Domain'].map(mapping).to_list()
docs = df['clean_readme'].astype('str').to_list()
vectorizer_model = CountVectorizer(stop_words="english")
# Create a fully supervised BERTopic instance
topic_model= BERTopic(
        vectorizer_model=vectorizer_model
)
topics, probs = topic_model.fit_transform(docs, y=y)

In [7]:
topics_per_class = topic_model.topics_per_class(docs, classes=df['Domain'].to_list())

In [9]:
topics_per_class

,Topic,Words,Frequency,Class
0,-1,"use, using, string, version, data",451,Non-web libraries and frameworks
1,0,"cjdns, cjdroute, layman, conf, sudo",1,Non-web libraries and frameworks
2,1,"view, animation, views, carousel, controller",177,Non-web libraries and frameworks
3,2,"license, view, android, layout, set",109,Non-web libraries and frameworks
4,3,"webpack, blocks, rebass, webpackblocks, define...",2,Non-web libraries and frameworks
...,...,...,...,...
161,50,"tpc, httpsgithubcomeishayjvmserializerswiki, r...",1,Documentation
162,63,"showcase, mvc, spring, cast, springone",1,Documentation
163,70,"cmedium, ceasy, challenge, solution, chard",14,Documentation
164,71,"angular, angularjs, angular2, components, appl...",12,Documentation


In [10]:
classes = topics_per_class['Class'].unique()

words_per_class = {}
for Class in classes:
    aux = topics_per_class[topics_per_class['Class'] == Class]
    aux_list = []
    for i in aux.index:
        aux_list.extend(aux['Words'])
    words_per_class[Class] = aux_list


In [11]:
words_per_class

{'Non-web libraries and frameworks': ['use, using, string, version, data',
  'cjdns, cjdroute, layman, conf, sudo',
  'view, animation, views, carousel, controller',
  'license, view, android, layout, set',
  'webpack, blocks, rebass, webpackblocks, defineconstants',
  'android, retrolambda, job, scaloid, blackbox',
  'neural, training, network, model, gpu',
  'string, grunt, ionic, padstr, characters',
  'rails, gem, activerecord, capybara, model',
  'image, cache, images, filter, 00',
  'framework, swift, xcode, irate, objectivec',
  'roles, entrust, role, permissions, permission',
  'python, tqdm, codeblock, target, alt',
  'vimplug, ycm, spf13vim, 92, vim',
  'helium, httpheliumfloatscom, floating, translucency, unidentified',
  'script, searchreplacedb, backups, srdbcliphp, db',
  'excel, format, worksheet, formats, cell',
  'bot, jieba, goose, chinese, botkit',
  'image, license, zoom, android, blend',
  'qlmarkdown, discount, quicklook, markdown, mdk',
  'kryo, json, serializers

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [15]:
class_embeddings = {}
for k in words_per_class.keys():
    class_embedding = []
    for topic in words_per_class[k]:
        topic_embedding = model.encode(topic)
        class_embedding.append(topic_embedding)
    class_embeddings[k] = class_embedding

In [18]:
len(class_embeddings['Non-web libraries and frameworks'][0])

384

In [20]:
for k in class_embeddings.keys():
    print(k, len(class_embeddings[k]))

Non-web libraries and frameworks 1849
Software tools 1089
Web libraries and frameworks 1296
Application software 441
System software 225
Documentation 324


In [21]:
1849 + 1089 + 1296 + 441 + 225 + 324

5224

In [ ]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):

    def __init__(self, labels_file, text_file):
        self.data_labels = pd.read_csv(labels_file)
        self.data_samples = pd.read_csv(text_file)

    def __len__(self):
        return len(self.data_labels)

    def __getitem__(self, idx):
        sample = self.data_labels.iloc[idx]
        label = self.data_labels.iloc[idx]

        return sample, label, idx

In [82]:
from torch.utils.data import DataLoader

training_data = np.fromfile('/home/panchoz/Documents/github/data/X_train_embeddings.dat', dtype=float)
test_data = np.fromfile('/home/panchoz/Documents/github/data/X_test_embeddings.dat', dtype=float)


train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

In [83]:
it = iter(train_dataloader)

In [84]:
it

In [85]:
first = next(it)

In [86]:
first

tensor([ 1.9027e-14,  2.1098e-12, -3.0496e-15,  1.6840e-11, -4.6404e-14,
        -5.3243e-21, -1.9255e-12, -8.7419e-11, -3.4889e-13,  1.3159e-14,
        -1.6872e-16, -1.1695e-18,  6.0790e-10, -1.3519e-12, -6.7119e-15,
         1.5647e-24,  2.9113e-23, -2.0690e-14,  6.9376e-12, -1.1297e-14,
         1.0917e-14, -9.5238e-15, -8.8365e-14, -3.5818e-15,  1.6776e-10,
        -4.7286e-11, -1.1477e-14,  8.0691e-16, -3.5886e-12, -8.6640e-19,
        -3.4237e-11,  4.2251e-18], dtype=torch.float64)

In [89]:
training_data.shape

(844416,)

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, x_file, y_file):
        self.x_data = np.load(x_file)
        self.y_data = pd.read_csv(y_file, header=None, names=['label'])

    def __len__(self):
        return len(self.y_data)

    def __getitem__(self, idx):
        x = torch.Tensor(self.x_data[idx])
        y = torch.tensor(self.y_data.iloc[idx, 0])
        return x, y

train_dataset = CustomDataset('X_train.np', 'y_train.csv')
test_dataset = CustomDataset('X_test.np', 'y_test.csv')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
